In [3]:
from typing import Any, Sequence, TypeVar, Generic, get_args, get_origin


T = TypeVar("T")
V = TypeVar("V")
U = TypeVar("U")
X = TypeVar("X")
Y = TypeVar("Y")


class BaseClass(Generic[T, U, V, X, Y]):
    pass


Z = TypeVar("Z")


class SecondBaseClass(Generic[Z]):
    pass


class ChildClass(BaseClass[T, str, V, X, dict]):
    pass


W = TypeVar("W")


class GrandChildClass(ChildClass[T, W, list], SecondBaseClass):
    pass
    

class GrandGrandChildClass(GrandChildClass[float, int]):
    pass


In [18]:
from typing import Union


def get_types(cls: type, superclass: type) -> Sequence[type]:
    def is_not_type_var(object: Any) -> bool:
        return type(object) != TypeVar

    def types_of_base(orig_bases: Any) -> Sequence[Union[type, TypeVar]]:
        return list(get_args(orig_bases))

    def is_eligible_subclass(superclass: type, parent: type):
        return hasattr(parent, '__orig_bases__') and issubclass(parent, superclass)
        
    type_list: list[type] = types_of_base(superclass.__orig_bases__[0])
    for parent in (p for p in reversed(cls.__mro__) if is_eligible_subclass(superclass, p)):
        for base in parent.__orig_bases__:
            origin = get_origin(base)
            if origin is None:
                continue
            current_types = types_of_base(base)
            types_set = 0
            for current_index, current_type in enumerate(current_types):
                if is_not_type_var(current_type):
                    offset = sum(is_not_type_var(t) for t in type_list[:current_index+1])
                    type_index = current_index + offset - types_set
                    type_list[type_index] = current_type
                    types_set += 1
    return type_list

get_types(GrandGrandChildClass, BaseClass)

[float, str, int, list, dict]